# Laboratorio 6
## Hunting with Data Science

---

Credo por:

- Juan Fernando De Leon Quezada
- Cristopher Barrios

In [13]:
# Librerias

import json
import pandas as pd
from features import domain_tld_extract

1. Cargue la información del archivo large_even.json en una lista, muestre la cantidad de registros total (deben ser 746, 909).

In [14]:
# Load data

json_data = open("large_eve_json\large_eve.json")

data_array = []
for i in json_data.readlines():
    if i:
        data_array.append(json.loads(i))

json_data.close()

print("Total Data: ", len(data_array))

Total Data:  746909


2. Debido a que estamos buscando dominios DGA, del total de registros, solamente estamos interesados en los registros DNS. Cargue únicamente aquellos registros en cuya llave se encuentra “DNS”.

* Muestre la nueva cantidad de registros filtrados.

In [15]:
# DNS data

dns = []

for i in data_array:
    if "dns" in i.keys():
        dns.append(i)

print("Total DNS data: ", len(dns))

Total DNS data:  15749


* Muestre la información de 2 registros cualesquiera.

In [16]:
print("{}\n{}".format(dns[0], dns[1]))

{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}
{'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}


* Debido a que la data consiste en json anidados, utilice la característica json_normalize para normalizar la información y asignarla en un dataframe.

In [17]:
# JSON Normalize

df = pd.json_normalize(dns)
df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


* Como estamos buscando dominios DGA, debemos filtrar los registros DNS para aquellos registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio), se debió bajar la cantidad a 2849 registros.

In [18]:
# DGA registers

a_reg = df[df['dns.rrtype'] == 'A']
a_reg['dns.rrtype'].value_counts()

A    2849
Name: dns.rrtype, dtype: int64

* Filtre los dominios únicos.

In [19]:
# Unique

unique = a_reg.drop_duplicates(subset=['dns.rrname'])
unique.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN


In [20]:
print(len(unique))

177


* Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando Flare como una columna nueva.

In [23]:
# Domain TLD Extract

unique['tld_extract_domain'] = unique['dns.rrname'].apply(domain_tld_extract)
unique.head()

C:\Users\jfdel\AppData\Local\Temp\ipykernel_32912\2081463357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique['tld_extract_domain'] = unique['dns.rrname'].apply(domain_tld_extract)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld_extract_domain
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,dropbox.com
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN,mozilla.com
